In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../..")) #make the folder "automl" part of this

# Load Model

In [ ]:
from automl.external_support.sb3.sb3_model_wrapper import SB3WrapperTorch

original_model = SB3WrapperTorch({"sb3_model" : "dqn-MountainCar-v0", "device" : "cpu"})
#loaded_model = SB3WrapperTorch({"sb3_model" : "dqn-CartPole-v1", "device" : "cpu"})

In [ ]:
original_model_name = "sb3_MountainCar_dqn"
#original_model_name = "sb3_CartPole_dqn"
model_name = original_model_name

In [ ]:
loaded_model = original_model.clone()

# Perturb Models

## Partially forget parameters (reinitialize them)

In [ ]:
percentage_to_forget = 100
std_of_reinit = 0.00000000000000001

In [ ]:
from automl.ml.models.torch_model_utils import perturb_model_parameters_partial_forgetting
perturb_model_parameters_partial_forgetting(loaded_model, fraction=percentage_to_forget / 100.0, std=std_of_reinit)

print(loaded_model.values)

In [ ]:
model_name = f"{model_name}_forgotten_{percentage_to_forget}_{std_of_reinit}"


## Perturb models randomly by percentage

In [ ]:
max_percentage = 5
min_percentage = 0

In [ ]:
from automl.ml.models.torch_model_utils import perturb_model_parameters


perturb_model_parameters(loaded_model, min_percentage / 100.0, max_percentage / 100.0)

print(loaded_model.values)

In [ ]:

model_name = f"{model_name}_perturbed_{min_percentage}_{max_percentage}"



## Perturb model with gaussian noise

In [ ]:
avg = 0
std = 0.3
fraction = 0.8

In [ ]:
from automl.ml.models.torch_model_utils import perturb_model_parameters_gaussian

perturb_model_parameters_gaussian(loaded_model, avg, std, fraction)

print(loaded_model.values)

In [ ]:
model_name = f"{model_name}_gaussian_{avg}_{std}_{fraction}"


# Study and save model

## Note the difference between the models

In [ ]:
from automl.ml.models.torch_model_utils import model_parameter_distance

l2_distance, avg_distance, cosine_sim = model_parameter_distance(original_model, loaded_model)

print(f"L2 distance between original and perturbed model: {l2_distance}")
print(f"Average distance between original and perturbed model: {avg_distance}")
print(f"Cosine similarity between original and perturbed model: {cosine_sim}")

## Save Model

In [ ]:
models_path = "data\\models"
specific_model_path = original_model_name

model_store_path = f"{models_path}\\{specific_model_path}"


model_file_name = f"{model_name}.pkl"

In [ ]:
import pickle
import os

from automl.utils.files_utils import new_path_if_exists

os.makedirs(model_store_path, exist_ok=True)
model_path = new_path_if_exists(model_file_name, dir=model_store_path)


with open(model_path, "wb") as f:
    pickle.dump(loaded_model, f)